# Classification showing and telling with pytorch

In [1]:
#deal with tensors
import torch   

#handling text data
from torchtext import data    

In [2]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True   

In [3]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

fields = [(None, None), ('text',TEXT),('label', LABEL)]


#loading custom dataset
training_data=data.TabularDataset(path = 'datasets/train_datasets.csv',format = 'csv',fields = fields,skip_header = True)

#print preprocessed text
print(vars(training_data.examples[0]))

{'text': ['walked', 'forest', 'already', 'fall', 'getting', 'cold']}


In [13]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [14]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi) 

BadZipFile: File is not a zip file

In [5]:
from torchtext import data # torchtext.data 임포트

# 필드 정의

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=20)


from torchtext.data import TabularDataset

train_data, test_data = TabularDataset.splits(
        path='datasets/', train='train_datasets.csv', test='test_datasets.csv', format='csv',
        fields=[('label', LABEL), ('text', TEXT)], skip_header=True)


print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

#ref: https://wikidocs.net/60314

훈련 샘플의 개수 : 205
테스트 샘플의 개수 : 51


In [6]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [12]:
#initialize glove embeddings

TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.50d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   

BadZipFile: File is not a zip file

In [5]:
import random
train_data, valid_data = train_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [5]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3, vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)  

BadZipFile: File is not a zip file

In [ ]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 16

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [ ]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [ ]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [ ]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

In [ ]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
#load weights
path='/content/saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();

#inference 
import spacy
nlp = spacy.load('en')

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence 
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #convert to integer sequence
    length = [len(indexed)]                                    #compute no. of words
    tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
    tensor = tensor.unsqueeze(1).T                             #reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)                   #convert to tensor
    prediction = model(tensor, length_tensor)                  #prediction 
    return prediction.item()                                   

In [ ]:
#make predictions
predict(model, "Are there any sports that you don't like?")

#insincere question
predict(model, "Why Indian girls go crazy about marrying Shri. Rahul Gandhi ji?")

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('data/showingTelling_csv.csv', delimiter = ',')

data.index.name = "index"
data.columns = ["type", "text"]
data

,type,text
index,,
0,1,The house was creepy.
1,1,I heard footsteps creeping behind me and it ma...
2,1,She was my best friend. I could tell her almos...
3,1,She hated it there because it smelled bad.
4,1,When they embraced she could tell he had been ...
...,...,...
253,2,"Her hand reached for the massive, iron door ha..."
254,2,The way the door decisively slammed behind her...
255,2,Dust coated every last surface. He ran his fin...


In [2]:
def shuffle(df, n=1, axis=0): #데이터가 1111 0000이라 잘 섞어주자. 
    df = data.copy()
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
    return df

shuffle(data)

train, test = train_test_split(data, test_size=0.2)
print(len(train))
print(len(test))

206
52


In [3]:
train.head() # 잘 섞였군만! 하지만 데이터 전처리가 필요하다.

,type,text
index,,
83,1,"Tiffany felt afraid. He pounded on the door, d..."
222,2,For ten or fifteen minutes Elroy held a course...
39,1,"I woke up and wanted to be active today. Then,..."
125,2,"championships, sings lead vocals in a rock ban..."
205,2,"₩gOh my God, I am five minutes late!₩h"


In [4]:
test.head()

,type,text
index,,
9,1,I had a great conversation with Tim over dinne...
229,2,"AND THEN, one day all foreign Jews were expell..."
191,2,Mushrooms and pepperoni sausage were layered t...
27,1,Dominique gives candy to her customers.
11,1,She was so angry that she threw a rock into th...


In [5]:
import re
import pandas
import numpy
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [6]:
def preprocessing(review, remove_stopwords=False):
        
    # 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review)

    # 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()

    if remove_stopwords: 
        # 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 단어 리스트를 공백을 넣어서 하나의 글로 합친다.
        clean_review = ' '.join(words)

    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review

In [7]:
clean_train_ = []
for review in train['text']:
    clean_train_.append(preprocessing(review, remove_stopwords=True))

# 전처리된 데이터 확인. 잘됨 !! ㅎ
clean_train_[0]

'tiffany felt afraid pounded door demanding let'

In [8]:
clean_train_[:5]

['tiffany felt afraid pounded door demanding let',
 'ten fifteen minutes elroy held course upstream river choppy silver gray turned straight north put engine full throttle felt bow lift beneath remember wind ears',
 'woke wanted active today heard amy cooking downstairs kitchen making lots noise kitchenware',
 'championships sings lead vocals rock band speaks five languages',
 'goh god five minutes late h']

In [9]:
clean_test_ = []
for review in test['text']:
    clean_test_.append(preprocessing(review, remove_stopwords=True))
    
# 전처리된 데이터 확인. 잘됨 !! ㅎ
clean_test_[0]

'great conversation tim dinner loved hearing stories'

In [10]:
clean_test_[:5]

['great conversation tim dinner loved hearing stories',
 'one day foreign jews expelled sighet moishe beadle foreigner crammed cattle cars hungarian police cried silently standing station platform crying',
 'mushrooms pepperoni sausage layered thickly top one another white mozzarella cheese bubbled bright red tomato sauce',
 'dominique gives candy customers',
 'angry threw rock house broke glass']

In [11]:
print(train["type"]) #showing telling 컬럽값을 확인해보고

index
83     1
222    2
39     1
125    2
205    2
      ..
107    1
68     1
2      1
76     1
199    2
Name: type, Length: 206, dtype: int64


In [12]:
train['cleaned_text'] = clean_train_ # 이제 전처리된 내용을 한눈에 비교해 볼 수 있다.
train[:5] #데이터 앞부분 5개반 확인해보자

/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,type,text,cleaned_text
index,,,
83,1,"Tiffany felt afraid. He pounded on the door, d...",tiffany felt afraid pounded door demanding let
222,2,For ten or fifteen minutes Elroy held a course...,ten fifteen minutes elroy held course upstream...
39,1,"I woke up and wanted to be active today. Then,...",woke wanted active today heard amy cooking dow...
125,2,"championships, sings lead vocals in a rock ban...",championships sings lead vocals rock band spea...
205,2,"₩gOh my God, I am five minutes late!₩h",goh god five minutes late h


In [13]:
test['cleaned_text'] = clean_test_
test[:5] #test 데이터셋도 전처리된 결과를 확인할 수 있다.

/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,type,text,cleaned_text
index,,,
9,1,I had a great conversation with Tim over dinne...,great conversation tim dinner loved hearing st...
229,2,"AND THEN, one day all foreign Jews were expell...",one day foreign jews expelled sighet moishe be...
191,2,Mushrooms and pepperoni sausage were layered t...,mushrooms pepperoni sausage layered thickly to...
27,1,Dominique gives candy to her customers.,dominique gives candy customers
11,1,She was so angry that she threw a rock into th...,angry threw rock house broke glass


In [50]:
train_dataset = train[['type', 'cleaned_text']] #전처리 1차 끝!
train_dataset[:5]

,type,cleaned_text
index,,
83,1,tiffany felt afraid pounded door demanding let
222,2,ten fifteen minutes elroy held course upstream...
39,1,woke wanted active today heard amy cooking dow...
125,2,championships sings lead vocals rock band spea...
205,2,goh god five minutes late h


In [49]:
test_dataset = test[['type', 'cleaned_text']]  #전처리 1차 끝!
test_dataset.head()

,type,cleaned_text
index,,
9,1,great conversation tim dinner loved hearing st...
229,2,one day foreign jews expelled sighet moishe be...
191,2,mushrooms pepperoni sausage layered thickly to...
27,1,dominique gives candy customers
11,1,angry threw rock house broke glass


df_test = test_dataset.astype(str)
df_test.dtypes

df_train = train_dataset.astype(str)
df_train.dtypes

train_dataset.values #train_dataset의 df를 numpy array로 변환

test_dataset.values #역시 이것도 변환

In [67]:
train_dataset.to_csv('datasets/train_datasets.csv', index=False, header=False, sep=',')
test_dataset.to_csv('datasets/test_datasets.csv', index=False, header=False, sep=',')

In [68]:
from torchtext import data # torchtext.data 임포트

# 필드 정의

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=20)


from torchtext.data import TabularDataset


train_data, test_data = TabularDataset.splits(
        path='datasets/', train='train_datasets.csv', test='test_datasets.csv', format='csv',
        fields=[('label', LABEL), ('text', TEXT)], skip_header=True)


print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

#ref: https://wikidocs.net/60314

훈련 샘플의 개수 : 205
테스트 샘플의 개수 : 51


In [69]:
print(vars(train_data[2])) # text, label이 구분됨

{'label': '2', 'text': ['championships', 'sings', 'lead', 'vocals', 'rock', 'band', 'speaks', 'five', 'languages']}


In [93]:
 #단어장 생성
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)



#단어장 생성 확인
#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   


#ref :::::: https://www.analyticsvidhya.com/blog/2020/01/first-text-classification-in-pytorch/


Size of TEXT vocabulary: 1542
Size of LABEL vocabulary: 3
[('h', 21), ('ft', 21), ('fs', 18), ('could', 15), ('like', 14), ('behind', 12), ('one', 12), ('cold', 11), ('face', 10), ('looked', 10)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x1a4b03a110>>, {'<unk>': 0, '<pad>': 1, 'ft': 2, 'h': 3, 'fs': 4, 'could': 5, 'like': 6, 'behind': 7, 'one': 8, 'cold': 9, 'door': 10, 'face': 11, 'long': 12, 'looked': 13, 'see': 14, 'felt': 15, 'garden': 16, 'never': 17, 'said': 18, 'black': 19, 'c': 20, 'first': 21, 'hands': 22, 'night': 23, 'sun': 24, 'time': 25, 'day': 26, 'eyes': 27, 'front': 28, 'got': 29, 'hand': 30, 'late': 31, 'little': 32, 'stairs': 33, 'walked': 34, 'would': 35, 'back': 36, 'blood': 37, 'breath': 38, 'came': 39, 'every': 40, 'f': 41, 'far': 42, 'father': 43, 'house': 44, 'knew': 45, 'light': 46, 'made': 47, 'man': 48, 'nervous': 49, 'old': 50, 'reached': 51, 'red': 52, 'saw': 53, 'sweat': 54, 'tree': 55, 'two': 56, 'us': 57, 'vi

In [71]:
print(train_data.fields.items()) # tex, label 로 구분되어 있는 것을 확인할 수 있다.

dict_items([('label', <torchtext.data.field.Field object at 0x1a4ae03490>), ('text', <torchtext.data.field.Field object at 0x1a4ae034d0>)])


In [95]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 16

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [96]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [97]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [100]:
#architecture
print(model)



classifier(
  (embedding): Embedding(1542, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)


In [101]:
#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

The model has 213,657 trainable parameters


TypeError: copy_(): argument 'other' (position 1) must be Tensor, not NoneType

In [72]:
print(test_data.fields.items()) # label, text가 각각 잘 구분되어 플드에 저작외었다. 이것을 이제 모델이 처 넣으면 된다.

dict_items([('label', <torchtext.data.field.Field object at 0x1a4ae03490>), ('text', <torchtext.data.field.Field object at 0x1a4ae034d0>)])


In [73]:
TEXT.build_vocab(train_data, min_freq=1, max_size=10000) #보캐브러리를 빌드한다. 그런데 왜하는겨?

In [74]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab))) #여기에 사용한 다어는 이많큼이다.

단어 집합의 크기 : 1542


In [75]:
print(TEXT.vocab.stoi) # 생성된 집합 내 단어들을 확인해보자

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x1a4ae032d0>>, {'<unk>': 0, '<pad>': 1, 'ft': 2, 'h': 3, 'fs': 4, 'could': 5, 'like': 6, 'behind': 7, 'one': 8, 'cold': 9, 'door': 10, 'face': 11, 'long': 12, 'looked': 13, 'see': 14, 'felt': 15, 'garden': 16, 'never': 17, 'said': 18, 'black': 19, 'c': 20, 'first': 21, 'hands': 22, 'night': 23, 'sun': 24, 'time': 25, 'day': 26, 'eyes': 27, 'front': 28, 'got': 29, 'hand': 30, 'late': 31, 'little': 32, 'stairs': 33, 'walked': 34, 'would': 35, 'back': 36, 'blood': 37, 'breath': 38, 'came': 39, 'every': 40, 'f': 41, 'far': 42, 'father': 43, 'house': 44, 'knew': 45, 'light': 46, 'made': 47, 'man': 48, 'nervous': 49, 'old': 50, 'reached': 51, 'red': 52, 'saw': 53, 'sweat': 54, 'tree': 55, 'two': 56, 'us': 57, 'village': 58, 'wind': 59, 'window': 60, 'already': 61, 'another': 62, 'away': 63, 'beautiful': 64, 'clouds': 65, 'dark': 66, 'darkness': 67, 'engine': 68, 'feet': 69, 'give': 70, 'glass': 71, 'go': 

In [44]:
#여기까지 잘됨, 이것을 이제 레이블, torch tensor로 변환 해야함. train_dataset, test_dataset 각각 처리할것

In [76]:
#토치텍스트의 테이터로더 생성

from torchtext.data import Iterator

batch_size = 16

In [77]:
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 13
테스트 데이터의 미니 배치 수 : 4


In [78]:
train_loader

In [79]:
type(train_loader)

torchtext.data.iterator.Iterator

In [80]:
batch = next(iter(train_loader)) # 16개씩 묶어줬음. 첫번째 미니배치에 저장

print(batch.text) #첫번째 미니 배치의 text 필드를 호출해서 확인해봄

tensor([[ 771, 1401,   72,   41,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  86,  232,  110,  232,  188,  876,  900,  980, 1431,  634,   37,  132,
          295,   15,   54,  433,    9,  824,    1,    1],
        [ 426,   33,  361,  310,  965,  282, 1257,  153,  353, 1194,   28,   10,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  23,    9, 1065, 1325,  382,  319,  331,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 579,  131,  312,  751, 1297, 1361,  102,   22,  933,    7,   36, 1017,
          975, 1380,  236,  721,  136,  191, 1317,  301],
        [  34,   33,   51,  183,  113,  979, 1077,   47,  140, 1008, 1284,  778,
          814,  633, 1237,   95,  725,  215,  826,   38],
        [1457,  808,  529,   90,  563,  627,  233,   22,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 203,  318,  218, 

In [81]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text[0]) # 첫번째 미니배치 중 첫번째 샘플

tensor([ 133,  819, 1381,  474,   55,  690,   63,  119,  588,  746,  962,    8,
        1131,  472,   55,  474, 1258,    6,  104,  449])


In [82]:
print(type(batch)) #미니배치 자료형 확인. 토치텍스늬 데이터로더는  'torchtext.data.batch.Batch'라는 객체를 가져온다. 

<class 'torchtext.data.batch.Batch'>


In [83]:
batch_tr = batch.text.numpy() #tensor를 array로 전환해봄(테스트) , 여기에 라벨값이 포함되어 있는데, 이것을 구분해서 추가해줘보자...
batch_tr[0]

array([ 133,  819, 1381,  474,   55,  690,   63,  119,  588,  746,  962,
          8, 1131,  472,   55,  474, 1258,    6,  104,  449])

In [84]:
# 여기가지 진행완료!!!

data_verif = []
data_verif = test_dataset
data_verif_len = len(data_verif)
data_verif_len

data_verif = []
data_verif = train_dataset
data_verif_len = len(data_verif)
data_verif_len

type(data_verif)

data_verif[:2]

In [89]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [90]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

AttributeError: 'DataFrame' object has no attribute 'get_vocab'

In [54]:
#원본코드인데.. 이미 가공된 데이터셋을 ngrams 처리해서 불러오기 때문에 입력데이터를 dataset에 맞게 수정해야 한다.

import torch
import torchtext
from torchtext.datasets import text_classification

NGRAMS = 2

import os

if not os.path.isdir('./data2'):
    os.mkdir('./data2')
    
    
#text_classification.DATASETS의 구조를 보고 결과데이터를 어떻게 생성하는지 분석하거나 이하 학습코드를 분석    
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./data2', ngrams=NGRAMS, vocab=None) 
#ref : https://pytorch.org/text/datasets.html#ag-news



BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

120000lines [00:08, 14582.67lines/s]
120000lines [00:15, 7822.49lines/s]
7600lines [00:00, 8340.65lines/s]


In [55]:
test_dataset[10:16]

[(3,
  tensor([    131,       5,   23258,      27,    2922,     357,    2688,     769,
              814,      14,      32,      15,      16,       6,     131,       7,
              230,     293,     452,     836,    6438,      85,       2,      51,
            43647,       2,   24372,       4,   60885,      51,  281059,       2,
                0,       9,   21969,     115,       2,      51,  108539,       2,
            36279,       4,      11,      81,      31,      90,      39,   23258,
                6,      27,     357,    4090,    1698,      53,       5,     273,
              821,       3,    1507,       7,       3,    1473,    3049,       2,
             9821,   53919,  115850,   75043,   30252,  478273,  244818,     822,
             4291,      43,      44,      46,     296,    2486,    2022,    8893,
            23909,   49141,  381768,    9169,   19041,      89,     122,       0,
            43648,   24031,   69185,   94356, 1174101,       0,  450795,       0,
           

In [56]:
train_dataset[0:10]

[(2,
  tensor([    572,     564,       2,    2326,   49106,     150,      88,       3,
             1143,      14,      32,      15,      32,      16,  443749,       4,
              572,     499,      17,      10,  741769,       7,  468770,       4,
               52,    7019,    1050,     442,       2,   14341,     673,  141447,
           326092,   55044,    7887,     411,    9870,  628642,      43,      44,
              144,     145,  299709,  443750,   51274,     703,   14312,      23,
          1111134,  741770,  411508,  468771,    3779,   86384,  135944,  371666,
             4052])),
 (2,
  tensor([  55003,    1474,    1150,    1832,    7559,      14,      32,      15,
               32,      16,    1262,    1072,     436,   55003,     131,       4,
           142576,      33,       6,    8062,      12,     756,  475640,       9,
           991346,    3186,       8,       3,     698,     329,       4,      33,
             6764, 1040465,   13979,      11,     278,     483,   

In [57]:
data_verif = []
data_verif = test_dataset
data_verif_len = len(data_verif)
data_verif_len

7600

In [58]:
data_verif_train = []
data_verif_train = train_dataset
data_verif_tr_len = len(data_verif_train)
data_verif_len

7600

In [59]:
os.path.exists('./data')
os.listdir('./data')

['.DS_Store', 'showingTelling.xlsx', 'showingTelling_csv.csv']

In [60]:
os.getcwd()

'/Users/kimkwangil/Project/01EssayFitAI/showing_telling'

In [61]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [62]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [64]:
train_dataset[0]

(2,
 tensor([    572,     564,       2,    2326,   49106,     150,      88,       3,
            1143,      14,      32,      15,      32,      16,  443749,       4,
             572,     499,      17,      10,  741769,       7,  468770,       4,
              52,    7019,    1050,     442,       2,   14341,     673,  141447,
          326092,   55044,    7887,     411,    9870,  628642,      43,      44,
             144,     145,  299709,  443750,   51274,     703,   14312,      23,
         1111134,  741770,  411508,  468771,    3779,   86384,  135944,  371666,
            4052]))

In [65]:
type(train_dataset)

torchtext.datasets.text_classification.TextClassificationDataset

In [63]:
VOCAB_SIZE

1308844

In [48]:
NUN_CLASS

4

In [4]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [7]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [8]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 25 seconds
	Loss: 0.0260(train)	|	Acc: 84.8%(train)
	Loss: 0.0001(valid)	|	Acc: 90.5%(valid)
Epoch: 2  | time in 0 minutes, 24 seconds
	Loss: 0.0118(train)	|	Acc: 93.7%(train)
	Loss: 0.0000(valid)	|	Acc: 89.3%(valid)
Epoch: 3  | time in 0 minutes, 24 seconds
	Loss: 0.0068(train)	|	Acc: 96.4%(train)
	Loss: 0.0001(valid)	|	Acc: 90.6%(valid)
Epoch: 4  | time in 0 minutes, 25 seconds
	Loss: 0.0038(train)	|	Acc: 98.2%(train)
	Loss: 0.0000(valid)	|	Acc: 90.8%(valid)
Epoch: 5  | time in 0 minutes, 24 seconds
	Loss: 0.0022(train)	|	Acc: 99.1%(train)
	Loss: 0.0000(valid)	|	Acc: 91.3%(valid)


In [9]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0002(test)	|	Acc: 89.3%(test)


In [12]:

import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

In [13]:
print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news
